# BinWaves example in Cantabria (Validation)

### BinWaves Notebooks Overview

This Jupyter Notebook is the first of three in the **BinWaves** modeling workflow, following Cagigal et al., 2024 :

1. `BinWaves_Propagation.ipynb`  
2. `BinWaves_Reconstruction.ipynb`  
3. `BinWaves_Validation.ipynb`

---

#### Before You Start

Make sure you have:

- Installed the latest version of `bluemath-tk`:  
  ```bash
  pip install bluemath-tk

Before continuing, ensure you have **created and activated a Python environment**.

*** Other Required Packages ***
- `wavespectra` 
- `cartophy`
 
---

<details>
<summary><strong>📁 BinWaves_Propagation.ipynb</strong></summary>

This notebook constructs the **library of pre-run cases** for all **monochromatic wave systems**.

##### Requirements:

- A **bathymetry** file placed in the `outputs/` folder, in the correct format.

If you don't have a specific bathymetry file for your study area, you can:

- **Download GEBCO bathymetry data** (~400 m resolution):  [https://download.gebco.net/](https://download.gebco.net/)


</details>

---

<details>
<summary><strong>📁 BinWaves_Reconstruction.ipynb</strong></summary>

This notebook reconstructs **wave conditions** using **offshore directional wave spectra**.

##### Requirements:

- Offshore wave spectrum data (e.g., **CAWCR** or **ERA5** datasets).
  - CAWCR spectra can be downloaded using a helper script (e.g., Javi's code?).

- **NOTE**: Optionally, apply **satellite corrections** to the hindcast spectrum before running BinWaves using the `CalVal` notebook, which handles the required format conversions.

</details>

---

<details open>
<summary><strong>📁 BinWaves_Validation.ipynb</strong></summary>

This notebook performs **validation** using **wave buoy data**, if available.

##### Requirements:

- Wave buoy data in a format compatible with BinWaves (if available).
- Some wave buoy data can be freely downloaded from:  
  🌊 [https://www.ndbc.noaa.gov/](https://www.ndbc.noaa.gov/)

- **NOTE:** You can use the `NDBC_buoy_data.ipynb` notebook to:
  - Download the buoy data.
  - Convert it into the appropriate format for BinWaves.

</details>

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

# Define buoy locations dictionary with both UTM and WGS84 coordinates
# if not buoys available  just run plot_selected_bathy(bathy=bathy)
buoys = {
    "44088": (-74.8390, 36.6120),
    "44100": (-75.5930, 36.2580),
    "44086": (-75.4210, 36.0010),
    "44056": (-75.7140, 36.2000),
    "44095": (-75.3300, 35.7500),
    "41120": (-75.2850, 35.2580),
    "41025": (-75.4540, 35.0100),
    "41159": (-76.9440, 34.2110),
    "41110": (-77.7150, 34.1420),
    "41013": (-77.7640, 33.4410),
    "41108": (-78.0160, 33.7210),
}


def find_site_index(kp_coeffs, target_x, target_y, tolerance=1.0):
    """
    Find the site index in kp_coeffs that matches the target coordinates.

    Parameters:
    -----------
    kp_coeffs : xarray.Dataset
        The kp coefficients dataset
    target_x : float
        Target UTM x coordinate
    target_y : float
        Target UTM y coordinate
    tolerance : float
        Tolerance for coordinate matching (in meters)

    Returns:
    --------
    int
        The site index that matches the coordinates
    """
    # Get all site coordinates
    site_x = kp_coeffs.utm_x.values
    site_y = kp_coeffs.utm_y.values

    # Calculate distances to all sites
    distances = np.sqrt((site_x - target_x) ** 2 + (site_y - target_y) ** 2)

    # Find the site with minimum distance
    site_index = np.argmin(distances)

    # Check if the closest site is within tolerance
    if distances[site_index] > tolerance:
        print(f"Warning: Closest site is {distances[site_index]:.2f} meters away")

    return site_index


def load_buoy_data(buoy_id, year=None):
    """
    Load buoy data and return both the wave data and location coordinates.

    Parameters:
    -----------
    buoy_id : str
        The buoy ID (e.g., '44100')
    year : str, optional
        The year to filter data for (default: None)

    Returns:
    --------
    tuple
        (buoy_waves, buoy_location, kp_coeffs)
    """
    # Load buoy data
    buoy_waves = pd.read_pickle(
        f"inputs/buoy_{buoy_id}_bulk_parameters.pkl"
    ).sort_index()
    if year:
        buoy_waves = buoy_waves.loc[year]
    buoy_waves = buoy_waves.dropna(subset=["Hs_Buoy", "Tp_Buoy", "Dir_Buoy"])

    # Get buoy location (both UTM and WGS84)
    buoy_location = buoys[buoy_id]

    # Load kp coefficients
    kp_coeffs = xr.open_dataset("outputs/kp_coefficients.nc")

    # Find the correct site index
    site_index = find_site_index(kp_coeffs, buoy_location[0], buoy_location[1])

    # Select the correct site
    kp_coeffs = kp_coeffs.isel(site=[site_index])

    return buoy_waves, buoy_location, kp_coeffs


# Example usage:
buoy_id = "41013"  # Change this to use different buoys
buoy_waves, buoy_location, kp_coeffs = load_buoy_data(buoy_id, "2023")

# Print the coordinates for verification
print(f"Buoy {buoy_id} coordinates:")
print(f"WGS84: {buoy_location[0]}, {buoy_location[1]}")
print(f"\nSelected kp_coeffs site coordinates:")
print(f"{kp_coeffs.utm_x.values}, {kp_coeffs.utm_y.values}")

Buoy 41013 coordinates:
WGS84: -77.764, 33.441

Selected kp_coeffs site coordinates:
[-77.764], [33.441]


In [ ]:
# import pandas as pd
# import xarray as xr

# # Load buoy data and kps

# buoy_waves = pd.read_pickle("outputs/buoy_44088_bulk_parameters.pkl").sort_index().loc["2022"]

# kp_coeffs = xr.open_dataset("outputs/kp_coefficients.nc").isel(site=[-11])
# kp_coeffs

In [2]:
kp_coeffs.utm_x.values, kp_coeffs.utm_y.values

(array([-77.764]), array([33.441]))

## Offshore Spectrum

> ⚠️ **NOTE:** If satellite correction was applied, ensure that 'satellite_correction=True'. The default value is False.

In [ ]:
from utils.operations import transform_ERA5_spectrum

model_parameters = pd.read_csv("CASES/swan_cases.csv").to_dict(orient="list")

# Load interest spectra
offshore_spectra, offshore_spectra_case = transform_ERA5_spectrum(
    era5_spectrum=xr.open_dataset("inputs/superpoint_result_interpolated.nc"),
    subset_parameters=model_parameters,
    available_case_num=kp_coeffs.case_num.values,
    satellite_correction=True,
)
offshore_spectra_case

In [ ]:
import numpy as np
from bluemath_tk.waves.binwaves import reconstruc_spectra

# First, ensure unique times in offshore_spectra_case
_, unique_idx_offshore = np.unique(offshore_spectra_case.time, return_index=True)
offshore_spectra_case = offshore_spectra_case.isel(time=unique_idx_offshore)

# Then, ensure unique times in buoy_waves
buoy_waves = buoy_waves[~buoy_waves.index.duplicated(keep="first")]


reconstructed_onshore_spectra = reconstruc_spectra(
    offshore_spectra=offshore_spectra_case.sel(time=buoy_waves.index, method="nearest"),
    kp_coeffs=kp_coeffs,
    num_workers=8,
)
reconstructed_onshore_spectra

In [ ]:
# TODO: Check why for some years it fails due to NaNs in the reconstructed spectra
from utils.plotting import plot_wave_series

# First ensure all datasets have unique time values
_, unique_idx_recon = np.unique(reconstructed_onshore_spectra.time, return_index=True)
reconstructed_onshore_spectra = reconstructed_onshore_spectra.isel(
    time=unique_idx_recon
)

_, unique_idx_offshore = np.unique(offshore_spectra.time, return_index=True)
offshore_spectra = offshore_spectra.isel(time=unique_idx_offshore)

# Make sure buoy_waves has unique indices (if not already done)
buoy_waves = buoy_waves[~buoy_waves.index.duplicated(keep="first")]

# Now plot with the deduplicated datasets
plot_wave_series(
    buoy_data=buoy_waves,
    binwaves_data=reconstructed_onshore_spectra.sel(
        time=buoy_waves.index, method="nearest"
    )
    .rename({"kps": "efth"})
    .squeeze()
    .spec,
    offshore_data=offshore_spectra.sel(time=buoy_waves.index, method="nearest").spec,
    times=buoy_waves.index.values,
)